In [1]:
import pandas as pd 
import numpy as np
import glob
from tqdm import tqdm
import cv2
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import datasets, models, transforms
from torchvision.models import resnet50

from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import clear_output

## Google Drive

копирование файлов с google drive

Вместо google drive можно использовать те архиви которые были в задаче

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cp /content/drive/MyDrive/AI/hacks-ai/ufa/train_dataset_train.zip /content/train_dataset_train.zip

In [4]:
!cp /content/drive/MyDrive/AI/hacks-ai/ufa/test_dataset_test.zip /content/test_dataset_test.zip

In [5]:
!unzip /content/train_dataset_train.zip

Archive:  /content/train_dataset_train.zip
  inflating: train.csv               
  inflating: Уфа_baseline.ipynb   
   creating: train/
  inflating: train/220301070305_0e13309ae71ffc37ba629d19f46e0784.jpg  
  inflating: train/220301070430_df1a737a5abe3707b424fbe9d1d92300.jpg  
  inflating: train/220301070439_91576ea7fbb6567ff743e271662f9d06.jpg  
  inflating: train/220301070525_8e9cf12c82b056104cfd8868f5958c3e.jpg  
  inflating: train/220301070535_78bd475d54865dc7b1c356f3436def6e.jpg  
  inflating: train/220301070614_4c1a339dd5056cd9747f3b4693f27b1b.jpg  
  inflating: train/220301070625_1be30d27fcfc119ed8dee6037a9ab385.jpg  
  inflating: train/220301070852_6431af3f1d3517498811dfa67f8a30be.jpg  
  inflating: train/220301070914_9f40eac64f72c8d6ee0e7f4fa37dc4ba.jpg  
  inflating: train/220301071234_2e7ed5acd0a8e8682b28f6618e7694c9.jpg  
  inflating: train/220301071302_180763d363e88725cc11143fb6923c1c.jpg  
  inflating: train/220301071500_fc5d48ef20ecb259c53d05b7f7888c62.jpg  
  inflating:

In [6]:
!unzip /content/test_dataset_test.zip -d test

Archive:  /content/test_dataset_test.zip
  inflating: test/3632659670_30122517390_7940463214_5.jpg  
  inflating: test/5462217165_30427088370_59864713467_5.jpg  
  inflating: test/5876578244_631703851_72334554003_4.jpg  
  inflating: test/6231154328_42126152388_7531475869_7.jpg  
  inflating: test/7316157724_59214342485_43319353865_6.jpg  
  inflating: test/7689511839_47811058882_39977310154_5.jpg  
  inflating: test/8387933680_5192337188_33405370907_1.jpg  
  inflating: test/9259096884_2251720133_44072689872_8.jpg  
  inflating: test/9939918097_95262672304_5380914686_5.jpg  
  inflating: test/10942248620_77738287904_92317579456_8.jpg  
  inflating: test/11476252728_90946703888_83373466997_4.jpg  
  inflating: test/11758169966_65799840524_72283028069_1.jpg  
  inflating: test/12054928568_20130374464_8647095983_8.jpg  
  inflating: test/12219465515_74065178154_47483019939_5.jpg  
  inflating: test/13693932815_79942882458_6879120134_3.jpg  
  inflating: test/16798749835_98183227174_58337

In [7]:
!mv /content/test/40567458052_99667140804_20508599954_6.jpeg /content/test/40567458052_99667140804_20508599954_6.jpg

In [10]:
!cp /content/drive/MyDrive/AI/hacks-ai/ufa/train_fixed.csv /content/train_fixed.csv

## Transformers


In [8]:
!pip install hugsvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 419 kB 11.2 MB/s 
     |████████████████████████████████| 585 kB 63.6 MB/s 
     |████████████████████████████████| 4.4 MB 61.6 MB/s 
     |████████████████████████████████| 431 kB 74.5 MB/s 
     |████████████████████████████████| 596 kB 66.6 MB/s 
     |████████████████████████████████| 140 kB 26.2 MB/s 
     |████████████████████████████████| 1.1 MB 71.7 MB/s 
     |████████████████████████████████| 94 kB 2.7 MB/s 
     |████████████████████████████████| 144 kB 70.8 MB/s 
     |████████████████████████████████| 271 kB 75.4 MB/s 
     |████████████████████████████████| 101 kB 14.3 MB/s 
     |████████████████████████████████| 6.6 MB 49.9 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [11]:
!rm -rf train1

In [12]:
!mkdir train1

In [13]:
!mkdir train1/blured
#!mkdir train1/not_blured
!mkdir train1/no_garbage
!mkdir train1/garbage

In [16]:
import random
import shutil

In [14]:
# читаем датасет
data_df = pd.read_csv("/content/train_fixed.csv")

In [17]:
for index, row in data_df.iterrows():
    if row['class'] == 2:
      shutil.copyfile('/content/train/' + row['ID_img'], '/content/train1/blured/' + row['ID_img'])
    elif row['class'] == 1:
      shutil.copyfile('/content/train/' + row['ID_img'], '/content/train1/garbage/' + row['ID_img'])
    else:
      shutil.copyfile('/content/train/' + row['ID_img'], '/content/train1/no_garbage/' + row['ID_img'])

In [24]:
from hugsvision.dataio.VisionDataset import VisionDataset

train, test, id2label, label2id = VisionDataset.fromImageFolder(
  "/content/train1/",
  test_ratio   = 0.15,
  balanced     = True,
  augmentation = True,
)

Split Datasets...
Balance train dataset...
The less represented label in train as 89 occurrences
Size of train after balancing is 267
train_ds:  226
+---------+--------+---------+------------+-------+
| Dataset | blured | garbage | no_garbage | Total |
+---------+--------+---------+------------+-------+
|  Train  |   77   |   77    |     72     |  226  |
|  Test   |   12   |   12    |     17     |  41   |
+---------+--------+---------+------------+-------+


In [25]:
from hugsvision.nnet.VisionClassifierTrainer import VisionClassifierTrainer
from transformers import DeiTFeatureExtractor, DeiTForImageClassification

In [26]:
huggingface_model = "facebook/deit-base-distilled-patch16-224"

# Train the model
trainer = VisionClassifierTrainer(
	model_name   = 'deit_model',
	train      	 = train,
	test      	 = test,
	output_dir   = '/content/out/',
	max_epochs   = 20,
	batch_size   = 16, # On RTX 2080 Ti
    lr 		     = 0.0001,
	fp16	     = True,
	model = DeiTForImageClassification.from_pretrained(
	    huggingface_model,
	    num_labels = len(label2id),
	    label2id   = label2id,
	    id2label   = id2label
	),
	feature_extractor = DeiTFeatureExtractor.from_pretrained(
		huggingface_model,
	),
)

loading configuration file https://huggingface.co/facebook/deit-base-distilled-patch16-224/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/714ee60a09194779d2acdbf17d75f0cca867f496a85550c69b29fd6f66c8b4fa.a46ddd6a2d7a28e819ee5dfa80b7ccfbcb74b937148567c84d918781d35b3721
Model config DeiTConfig {
  "architectures": [
    "DeiTForImageClassificationWithTeacher"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "blured",
    "1": "garbage",
    "2": "no_garbage"
  },
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "blured": "0",
    "garbage": "1",
    "no_garbage": "2"
  },
  "layer_norm_eps": 1e-12,
  "model_type": "deit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": true,
  "transformers_version": "4.20.1"
}

loading weights fi

{'0': 'blured', '1': 'garbage', '2': 'no_garbage'}
{'blured': '0', 'garbage': '1', 'no_garbage': '2'}
Trainer builded!
Start Training!


Epoch,Training Loss,Validation Loss
1,No log,0.290243
2,No log,0.282283
3,No log,0.127596
4,No log,0.140563
5,No log,0.212672
6,No log,0.133998
7,No log,0.128886
8,No log,0.129826
9,No log,0.131254
10,No log,0.133289


***** Running Evaluation *****
  Num examples = 41
  Batch size = 16
***** Running Evaluation *****
  Num examples = 41
  Batch size = 16
***** Running Evaluation *****
  Num examples = 41
  Batch size = 16
***** Running Evaluation *****
  Num examples = 41
  Batch size = 16
***** Running Evaluation *****
  Num examples = 41
  Batch size = 16
***** Running Evaluation *****
  Num examples = 41
  Batch size = 16
***** Running Evaluation *****
  Num examples = 41
  Batch size = 16
***** Running Evaluation *****
  Num examples = 41
  Batch size = 16
***** Running Evaluation *****
  Num examples = 41
  Batch size = 16
***** Running Evaluation *****
  Num examples = 41
  Batch size = 16
***** Running Evaluation *****
  Num examples = 41
  Batch size = 16
***** Running Evaluation *****
  Num examples = 41
  Batch size = 16
***** Running Evaluation *****
  Num examples = 41
  Batch size = 16
***** Running Evaluation *****
  Num examples = 41
  Batch size = 16
***** Running Evaluation *****
  N

Model saved at: /content/out/DEIT_MODEL/20_2022-07-03-06-55-13


In [27]:
ref, hyp = trainer.evaluate_f1_score()

100%|██████████| 41/41 [00:01<00:00, 33.75it/s]

              precision    recall  f1-score   support

      blured     1.0000    1.0000    1.0000        12
     garbage     1.0000    0.9167    0.9565        12
  no_garbage     0.9444    1.0000    0.9714        17

    accuracy                         0.9756        41
   macro avg     0.9815    0.9722    0.9760        41
weighted avg     0.9770    0.9756    0.9754        41

Logs saved at: /content/out/DEIT_MODEL/20_2022-07-03-06-55-13


Модель сохраняется в отдельной папке, поэтому нужно поменять путь, можно взять из предыдущей ячейки новый путь

In [ ]:
!cp /content/out/DEIT_MODEL/20_2022-07-02-18-02-01/model/config.json /content/out/DEIT_MODEL/20_2022-07-02-18-02-01/model/preprocessor_config.json

In [ ]:
!mkdir /content/drive/MyDrive/AI/hacks-ai/ufa/deit_model/

In [ ]:
!cp /content/out/DEIT_MODEL/20_2022-07-02-18-02-01/model/config.json /content/drive/MyDrive/AI/hacks-ai/ufa/deit_model/config.json

In [ ]:
!cp /content/out/DEIT_MODEL/20_2022-07-02-18-02-01/model/pytorch_model.bin /content/drive/MyDrive/AI/hacks-ai/ufa/deit_model/pytorch_model.bin

In [ ]:
import os.path

path = "/content/out/DEIT_MODEL/20_2022-07-02-18-02-01/model"

classifier = VisionClassifierInference(
    feature_extractor = DeiTFeatureExtractor.from_pretrained(path),
    model = DeiTForImageClassification.from_pretrained(path),
)

loading feature extractor configuration file /content/out/DEIT_MODEL/20_2022-07-02-18-02-01/model/preprocessor_config.json
Feature extractor DeiTFeatureExtractor {
  "_name_or_path": "facebook/deit-base-distilled-patch16-224",
  "architectures": [
    "DeiTForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "crop_size": 224,
  "do_center_crop": true,
  "do_normalize": true,
  "do_resize": true,
  "encoder_stride": 16,
  "feature_extractor_type": "DeiTFeatureExtractor",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "blured",
    "1": "garbage",
    "2": "no_garbage"
  },
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_size": 224,
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "blured": "0",
    "garbage": "1",
    "no_garbage": "2"
  },
  "layer_norm_eps": 1e-12,
  "model_type": "deit",
  "num_attention_heads": 12,


Model loaded!


In [ ]:
label = classifier.predict(img_path='/content/test/11476252728_90946703888_83373466997_4.jpg')
print("Predicted class:", label)

Predicted class: garbage


In [ ]:
test_df = pd.read_csv("/content/sample_solution.csv")
test_df = test_df.drop(["class"], axis = 1)

In [ ]:
test_df.head(10)

,ID_img
0,34020749806_42065966214_42113475048_2
1,80128313599_98196458454_79029076007_8
2,17820331238_48919943775_53688855463_7
3,70492442702_21083599816_22777758696_0
4,94790217016_17108156014_60668676818_2
5,86587431345_19415917435_15848908132_5
6,72247588360_17217555187_90487891213_1
7,92025186647_49423511366_45537106756_8
8,11476252728_90946703888_83373466997_4
9,93794757271_68289030603_19125007053_7


#Предсказываем нашей моделью результат

In [ ]:

predicts = []

for idx, row in tqdm(test_df.iterrows()):
  img_path = '/content/test/' + row['ID_img'] + '.jpg';
  label = classifier.predict(img_path=img_path)
  if label == 'blured':
    predicts.append(2)
  elif label == 'garbage':
    predicts.append(1)
  else:
    predicts.append(0)


225it [02:14,  1.67it/s]


In [ ]:
len(predicts)

225

In [ ]:
test_df['class'] = predicts


In [ ]:
test_df

,ID_img,class
0,34020749806_42065966214_42113475048_2,2
1,80128313599_98196458454_79029076007_8,1
2,17820331238_48919943775_53688855463_7,2
3,70492442702_21083599816_22777758696_0,1
4,94790217016_17108156014_60668676818_2,2
...,...,...
220,60879177998_15763718934_82574532042_2,2
221,11758169966_65799840524_72283028069_1,1
222,9259096884_2251720133_44072689872_8,0
223,37732252922_9265441355_19052721018_3,2


In [ ]:
test_df.to_csv("submit.csv", index=False)